In [1]:
%cd /content/drive/MyDrive/final_data

/content/drive/MyDrive/final_data


In [2]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
headers = {"X-Naver-Client-Id": "lJrKhZKFXCGXSoYzh7f7",
           "X-Naver-Client-Secret":"YYPQvTkBxR"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
## 대분류 카테고리 설정
categories = [
      {"name": "모자", "param": [ "50000181"]},
      {"name": "벨트", "param": [ "50000180"]},
      {"name": "선글라스/안경테", "param": [ "50000183"]},
      {"name": "순금", "param": [ "50000432"]},
      {"name": "시계", "param": [ "50000425"]},
      {"name": "여성신발", "param": [ "50000173"]},
      {"name": "양말", "param": [ "50000166"]},
      {"name": "여행용가방/소품", "param": [ "50000178"]},
      {"name": "장갑", "param": [ "50000551"]},
      {"name": "주얼리", "param": [ "50000189"]},
      {"name": "지갑", "param": [ "50001471"]},
      {"name": "패션소품", "param": [ "50000567"]},
      {"name": "헤어액세서리", "param": [ "50000184"]}
  ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
data = {
  "startDate": "2021-05-10",
  "endDate": "2023-07-03",
  "timeUnit": "date",
  "category": [
      {"name": "패션잡화", "param": [ "50000001"]},
      {"name": "남성신발", "param": [ "50000174"]},
  ],
  "device":"",
  "gender":"",
  "ages":[]
}
## 데이터 정제 및 merge 파트

def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df

response = requests.post(url, headers=headers, json=data)

results = response.json()["results"]


df_a = change_index(pd.DataFrame(results[0]["data"]), "패션잡화")
df_b = change_index(pd.DataFrame(results[1]["data"]), "남성신발")
##
df_all = pd.merge(df_a, df_b, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
for category in categories:
    if len(data["category"]) > 2:
        data["category"].pop(2)
    data["category"].append(category)

    result = requests.post(url, headers=headers, json=data)

    results = result.json()["results"]

    df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

    df_all = pd.merge(df_all, df, left_on="period", right_on="period")

df_all.drop(["패션잡화"], axis=1, inplace=True)
df_all.to_excel("data_mid_10_all.xlsx", sheet_name="click ratio")
df_all.head()

,남성신발,모자,벨트,선글라스/안경테,순금,시계,여성신발,양말,여행용가방/소품,장갑,주얼리,지갑,패션소품,헤어액세서리
period,,,,,,,,,,,,,,
2021-05-10,12.02174,2.44833,0.54212,1.51041,0.04076,0.02012,11.63245,0.96218,0.61878,0.02954,5.01534,0.92643,0.01057,0.76299
2021-05-11,11.66945,2.47859,0.51372,1.50514,0.03580,0.02205,11.48585,0.94191,0.69697,0.03402,4.88670,0.85049,0.00886,0.76663
2021-05-12,11.46025,2.61627,0.52574,1.44108,0.03470,0.01893,11.86613,0.93551,0.56297,0.03904,4.89193,0.78601,0.00822,0.77020
2021-05-13,10.91306,2.79239,0.47355,1.46638,0.02908,0.01762,11.97745,0.93313,0.60418,0.04460,4.51802,0.73853,0.00889,0.79353
2021-05-14,10.01901,2.55165,0.42827,1.39799,0.02860,0.01774,11.74289,0.85126,0.53751,0.03827,4.36706,0.74264,0.00620,0.73147


In [3]:
categories = [
      {"name": "모자", "param": [ "50000181"]},
      {"name": "벨트", "param": [ "50000180"]},
      {"name": "선글라스/안경테", "param": [ "50000183"]},
      {"name": "순금", "param": [ "50000432"]},
      {"name": "시계", "param": [ "50000425"]},
      {"name": "여성신발", "param": [ "50000173"]},
      {"name": "양말", "param": [ "50000166"]},
      {"name": "여행용가방/소품", "param": [ "50000178"]},
      {"name": "장갑", "param": [ "50000551"]},
      {"name": "주얼리", "param": [ "50000189"]},
      {"name": "지갑", "param": [ "50001471"]},
      {"name": "패션소품", "param": [ "50000567"]},
      {"name": "헤어액세서리", "param": [ "50000184"]}
  ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
data = {
  "startDate": "2021-05-10",
  "endDate": "2023-07-03",
  "timeUnit": "date",
  "category": [
      {"name": "패션잡화", "param": [ "50000001"]},
      {"name": "남성신발", "param": [ "50000174"]},
  ],
  "device":"",
  "gender":"f",
  "ages":[]
}
## 데이터 정제 및 merge 파트

def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df

response = requests.post(url, headers=headers, json=data)

results = response.json()["results"]


df_a = change_index(pd.DataFrame(results[0]["data"]), "패션잡화")
df_b = change_index(pd.DataFrame(results[1]["data"]), "남성신발")
##
df_all = pd.merge(df_a, df_b, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
for category in categories:
    if len(data["category"]) > 2:
        data["category"].pop(2)
    data["category"].append(category)

    result = requests.post(url, headers=headers, json=data)

    results = result.json()["results"]

    df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

    df_all = pd.merge(df_all, df, left_on="period", right_on="period")

df_all.drop(["패션잡화"], axis=1, inplace=True)
df_all.to_excel("data_mid_10_f.xlsx", sheet_name="click ratio")
df_all.head()

,남성신발,모자,벨트,선글라스/안경테,순금,시계,여성신발,양말,여행용가방/소품,장갑,주얼리,지갑,패션소품,헤어액세서리
period,,,,,,,,,,,,,,
2021-05-10,7.99464,2.44600,0.41182,1.03378,0.04032,0.01195,16.78080,1.06667,0.59291,0.04021,6.38950,1.30428,0.01619,1.13662
2021-05-11,7.79660,2.46827,0.42463,1.07982,0.03358,0.01501,16.57859,1.01455,0.58932,0.04588,6.10003,1.18228,0.01369,1.15814
2021-05-12,7.76179,2.68186,0.42132,1.01885,0.03420,0.01285,17.22216,0.99746,0.55023,0.05500,6.18472,1.11414,0.01270,1.15900
2021-05-13,7.39246,3.04112,0.36223,1.05708,0.02828,0.01217,17.57169,0.97741,0.52180,0.06559,5.85993,1.02655,0.01478,1.21150
2021-05-14,6.74168,2.76990,0.33741,1.03053,0.02764,0.00994,17.26670,0.86268,0.45255,0.05442,5.62879,1.04677,0.00924,1.10034


In [4]:
categories = [
      {"name": "모자", "param": [ "50000181"]},
      {"name": "벨트", "param": [ "50000180"]},
      {"name": "선글라스/안경테", "param": [ "50000183"]},
      {"name": "순금", "param": [ "50000432"]},
      {"name": "시계", "param": [ "50000425"]},
      {"name": "여성신발", "param": [ "50000173"]},
      {"name": "양말", "param": [ "50000166"]},
      {"name": "여행용가방/소품", "param": [ "50000178"]},
      {"name": "장갑", "param": [ "50000551"]},
      {"name": "주얼리", "param": [ "50000189"]},
      {"name": "지갑", "param": [ "50001471"]},
      {"name": "패션소품", "param": [ "50000567"]},
      {"name": "헤어액세서리", "param": [ "50000184"]}
  ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
data = {
  "startDate": "2021-05-10",
  "endDate": "2023-07-03",
  "timeUnit": "date",
  "category": [
      {"name": "패션잡화", "param": [ "50000001"]},
      {"name": "남성신발", "param": [ "50000174"]},
  ],
  "device":"",
  "gender":"m",
  "ages":[]
}
## 데이터 정제 및 merge 파트

def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df

response = requests.post(url, headers=headers, json=data)

results = response.json()["results"]


df_a = change_index(pd.DataFrame(results[0]["data"]), "패션잡화")
df_b = change_index(pd.DataFrame(results[1]["data"]), "남성신발")
##
df_all = pd.merge(df_a, df_b, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
for category in categories:
    if len(data["category"]) > 2:
        data["category"].pop(2)
    data["category"].append(category)

    result = requests.post(url, headers=headers, json=data)

    results = result.json()["results"]

    df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

    df_all = pd.merge(df_all, df, left_on="period", right_on="period")

df_all.drop(["패션잡화"], axis=1, inplace=True)
df_all.to_excel("data_mid_10_m.xlsx", sheet_name="click ratio")
df_all.head()

,남성신발,모자,벨트,선글라스/안경테,순금,시계,여성신발,양말,여행용가방/소품,장갑,주얼리,지갑,패션소품,헤어액세서리
period,,,,,,,,,,,,,,
2021-05-10,33.31996,3.79595,1.32090,2.91786,0.08268,0.05641,8.18329,1.41285,0.70407,0.02281,4.10042,0.69630,0.00438,0.37560
2021-05-11,32.05347,3.75816,1.20292,2.85480,0.07910,0.05647,7.67453,1.39562,0.67348,0.02024,3.93198,0.62446,0.00369,0.33806
2021-05-12,31.18791,3.81601,1.24567,2.70404,0.06744,0.04607,7.79696,1.43448,0.67048,0.02501,3.80204,0.54705,0.00257,0.35423
2021-05-13,29.37412,3.66751,1.19741,2.75776,0.06049,0.03980,7.36330,1.43066,0.63098,0.02294,3.56158,0.52310,0.00112,0.34113
2021-05-14,27.13858,3.39327,1.03919,2.44854,0.06055,0.04394,7.20910,1.29745,0.53852,0.02118,3.21537,0.51658,0.00231,0.33592
